In [1]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split

In [ ]:
labels=pd.read_csv('/content/labels.csv')
train,valid=train_test_split(labels,train_size=0.8,shuffle=True,stratify=labels['breed'],random_state=42)

In [3]:
train,train_labels=train['id'].reset_index(drop=True),train['breed'].reset_index(drop=True)
val,val_labels=valid['id'].reset_index(drop=True),valid['breed'].reset_index(drop=True)

In [ ]:
breeds=dict()
breed_count=1

for breed in labels['breed'].value_counts().index:

    breeds[breed]=breed_count-1
    breed_count+=1

val_labels_torch=torch.zeros(len(val_labels),1)
train_labels_torch=torch.zeros(len(train_labels),1)

for index in val_labels.index:
    val_labels_torch[index]=breeds[val_labels.iloc[index]]

for index in train_labels.index:
    train_labels_torch[index]=breeds[train_labels.iloc[index]]

val_labels_torch = val_labels_torch.long()
train_labels_torch = train_labels_torch.long()


val_labels_torch[91]
# val_labels=torch.tensor(val_labels.values)
# train_labels=torch.tensor(train_labels.values)

tensor([21])

In [5]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor()
])


def create_dataset(series, directory_path, transform):
    tensors = []
    for i in range(len(series)):
        if(series[i].endswith('.jpg')):
            img_file=series[i]
        else:
            img_file = series[i] + '.jpg'
        img_path = os.path.join(directory_path, img_file)
        img = Image.open(img_path).convert('RGB')  # Ensure RGB
        img_t = transform(img)
        tensors.append(img_t)
    return torch.stack(tensors, dim=0)


In [6]:
directory_path=Path('/kaggle/input/dog-breed-identification/train')
train_dataset=create_dataset(train,directory_path,transform)

In [7]:


# For validation: no augmentation
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])
val_dataset = create_dataset(val, directory_path, transform=val_transform)


In [8]:
from torch.utils.data import TensorDataset

train_dataset_torch=TensorDataset(train_dataset,train_labels_torch)
val_dataset_torch=TensorDataset(val_dataset,val_labels_torch)

In [9]:
train_dataset.shape,train_labels_torch.shape

(torch.Size([8177, 3, 224, 224]), torch.Size([8177, 1]))

In [10]:
val_dataset_torch[0]

(tensor([[[0.2980, 0.2667, 0.1569,  ..., 0.2510, 0.0471, 0.0353],
          [0.2275, 0.1490, 0.2157,  ..., 0.1020, 0.0196, 0.0078],
          [0.2000, 0.1098, 0.2745,  ..., 0.0510, 0.0275, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1843,  ..., 0.1176, 0.1373, 0.1451],
          [0.1490, 0.1843, 0.1804,  ..., 0.1765, 0.1569, 0.1451],
          [0.2431, 0.2784, 0.2118,  ..., 0.1529, 0.1216, 0.1059]],
 
         [[0.3765, 0.3373, 0.2275,  ..., 0.2863, 0.0549, 0.0314],
          [0.3137, 0.2314, 0.2980,  ..., 0.1373, 0.0275, 0.0039],
          [0.2941, 0.2039, 0.3647,  ..., 0.0824, 0.0392, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1882,  ..., 0.1333, 0.1529, 0.1608],
          [0.1490, 0.1843, 0.1843,  ..., 0.1843, 0.1647, 0.1529],
          [0.2431, 0.2784, 0.2157,  ..., 0.1569, 0.1255, 0.1059]],
 
         [[0.3765, 0.3608, 0.2588,  ..., 0.2235, 0.0431, 0.0510],
          [0.2667, 0.2000, 0.2706,  ..., 0.0745, 0.0157, 0.0235],
          [0.1961, 0.1176, 0.2902,  ...,

In [11]:
from torchvision.models import resnet101, ResNet101_Weights
import torch.nn as nn
import torch.optim as optim

resnet = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
for param in resnet.parameters():
    param.requires_grad = False


resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),  # From ResNet's output to 256 units
    nn.ReLU(),                              # Activation function
    nn.BatchNorm1d(256),                    # Normalization for stability
    nn.Linear(256, 256),                    # Additional dense layer
    nn.ReLU(),                              # Activation again
    nn.Dropout(0.3),                        # Dropout to reduce overfitting
    nn.BatchNorm1d(256),                    # Another batch norm layer
    nn.Linear(256, 120)                     # Final layer: 120 output classes
)
model_torch = resnet

optimizier=optim.Adam(model_torch.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 193MB/s]


In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset_torch, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset_torch, batch_size=64, shuffle=False)

n_epochs=20
model_torch=model_torch.to('cuda')
for i in range(1,n_epochs+1):
    model_torch.train()
    training_loss=0.0
    for img,label in train_loader:
        img=img.to('cuda')
        label=label.to('cuda')
        label=label.squeeze(1)
        outputs=model_torch(img)
        optimizier.zero_grad()
        losses=loss_fn(outputs,label)
        training_loss+=losses.item()
        losses.backward()
        optimizier.step()

    model_torch.eval()
    val_loss=0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for img,label in val_loader:
            img=img.to('cuda')
            label=label.to('cuda')
            label=label.squeeze(1)
            outputs=model_torch(img)
            loss=loss_fn(outputs,label)
            val_loss+=loss.item()

            _, predicted = torch.max(outputs, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print(f"Epoch [{i}/{n_epochs}], Training Loss: {training_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch [1/20], Training Loss: 3.9497, Validation Loss: 3.0209, Accuracy: 58.58%
Epoch [2/20], Training Loss: 2.7095, Validation Loss: 2.2647, Accuracy: 69.83%
Epoch [3/20], Training Loss: 2.0879, Validation Loss: 1.7963, Accuracy: 75.55%
Epoch [4/20], Training Loss: 1.6815, Validation Loss: 1.5181, Accuracy: 78.19%
Epoch [5/20], Training Loss: 1.4135, Validation Loss: 1.3119, Accuracy: 79.76%
Epoch [6/20], Training Loss: 1.2001, Validation Loss: 1.1393, Accuracy: 80.49%
Epoch [7/20], Training Loss: 1.0472, Validation Loss: 1.0330, Accuracy: 81.37%
Epoch [8/20], Training Loss: 0.9352, Validation Loss: 0.9782, Accuracy: 81.52%
Epoch [9/20], Training Loss: 0.8287, Validation Loss: 0.9178, Accuracy: 81.17%
Epoch [10/20], Training Loss: 0.7464, Validation Loss: 0.8815, Accuracy: 81.37%
Epoch [11/20], Training Loss: 0.6702, Validation Loss: 0.8164, Accuracy: 81.81%
Epoch [12/20], Training Loss: 0.6268, Validation Loss: 0.8483, Accuracy: 80.88%
Epoch [13/20], Training Loss: 0.5622, Validation 

In [13]:
torch.save(resnet.state_dict(), 'resnet_custom_head.pth')


In [14]:
os.listdir('/kaggle/input/dog-breed-identification/test')[1:10]

['f7a32505c12649183c5991ecfa7d68b3.jpg',
 'bc6d50ffb4644feb34530aa58943e85b.jpg',
 '7cbc041e79135a572aad87904b5c9c57.jpg',
 'f3b603f10d6a344f0395fb46b242ff8e.jpg',
 'db55098d859d27a3c05b0e3fc41d43e7.jpg',
 '665bab06a3ea861c0812c778486988e3.jpg',
 '7b57911ea7559cebd6db308125926642.jpg',
 'df0d6ba158287cb2b3ed6459a22d42ba.jpg',
 'b80d89ad4a17ac2537ff46d251efcd7c.jpg']

In [15]:

test=os.listdir('/kaggle/input/dog-breed-identification/test')
test_dataset=create_dataset(test,'/kaggle/input/dog-breed-identification/test',val_transform)
test_dataset=test_dataset.to(device='cuda')

In [16]:
import pandas as pd
output=pd.DataFrame(columns=['file',*breeds.keys()])

In [17]:
output

,file,scottish_deerhound,maltese_dog,afghan_hound,entlebucher,bernese_mountain_dog,shih-tzu,pomeranian,great_pyrenees,basenji,...,otterhound,giant_schnauzer,walker_hound,tibetan_mastiff,german_shepherd,golden_retriever,komondor,brabancon_griffon,eskimo_dog,briard


In [18]:
breeds

{'scottish_deerhound': 0,
 'maltese_dog': 1,
 'afghan_hound': 2,
 'entlebucher': 3,
 'bernese_mountain_dog': 4,
 'shih-tzu': 5,
 'pomeranian': 6,
 'great_pyrenees': 7,
 'basenji': 8,
 'samoyed': 9,
 'tibetan_terrier': 10,
 'airedale': 11,
 'cairn': 12,
 'leonberg': 13,
 'japanese_spaniel': 14,
 'beagle': 15,
 'blenheim_spaniel': 16,
 'miniature_pinscher': 17,
 'australian_terrier': 18,
 'irish_wolfhound': 19,
 'saluki': 20,
 'lakeland_terrier': 21,
 'papillon': 22,
 'norwegian_elkhound': 23,
 'siberian_husky': 24,
 'whippet': 25,
 'pug': 26,
 'chow': 27,
 'italian_greyhound': 28,
 'pembroke': 29,
 'newfoundland': 30,
 'ibizan_hound': 31,
 'border_terrier': 32,
 'silky_terrier': 33,
 'lhasa': 34,
 'dandie_dinmont': 35,
 'bedlington_terrier': 36,
 'sealyham_terrier': 37,
 'rhodesian_ridgeback': 38,
 'irish_setter': 39,
 'boston_bull': 40,
 'old_english_sheepdog': 41,
 'collie': 42,
 'english_foxhound': 43,
 'african_hunting_dog': 44,
 'bouvier_des_flandres': 45,
 'schipperke': 46,
 'kelp

In [19]:
import torch.nn.functional as F
i=0
for file,sample in zip(os.listdir('/kaggle/input/dog-breed-identification/test'),test_dataset):
    sample=sample.cuda()
    output_values=model_torch(sample.unsqueeze(dim=0))
    output_values=output_values.cpu()
    probabilities = F.softmax(output_values, dim=1)
    output.loc[i]=[file,*probabilities[0].detach().numpy()]
    i+=1

In [20]:
output.head()

,file,scottish_deerhound,maltese_dog,afghan_hound,entlebucher,bernese_mountain_dog,shih-tzu,pomeranian,great_pyrenees,basenji,...,otterhound,giant_schnauzer,walker_hound,tibetan_mastiff,german_shepherd,golden_retriever,komondor,brabancon_griffon,eskimo_dog,briard
0,09e51e17e2b756ff2ace8a87bd1443fa.jpg,0.000007,0.000087,0.000216,0.000013,0.000007,0.000046,0.000063,0.000032,0.000228,...,0.000037,0.000166,0.000225,5.209325e-05,0.000192,0.000022,0.000104,0.000605,0.000032,0.000019
1,f7a32505c12649183c5991ecfa7d68b3.jpg,0.000009,0.000087,0.000012,0.000037,0.000009,0.000014,0.000017,0.000057,0.000165,...,0.000010,0.000007,0.000096,6.951765e-06,0.000008,0.000428,0.000002,0.000104,0.000026,0.000016
2,bc6d50ffb4644feb34530aa58943e85b.jpg,0.001802,0.000052,0.012416,0.000106,0.000082,0.000360,0.000389,0.000598,0.000060,...,0.000089,0.000127,0.000254,9.847712e-04,0.013357,0.052928,0.000654,0.000109,0.000430,0.000605
3,7cbc041e79135a572aad87904b5c9c57.jpg,0.000230,0.000003,0.000003,0.000010,0.000014,0.000003,0.000008,0.000014,0.000090,...,0.000004,0.000011,0.000016,8.130550e-07,0.000003,0.000001,0.000008,0.000005,0.000003,0.000001
4,f3b603f10d6a344f0395fb46b242ff8e.jpg,0.000011,0.000012,0.000010,0.000021,0.000054,0.000132,0.000011,0.000037,0.000219,...,0.000028,0.000021,0.000281,4.888405e-06,0.000004,0.000203,0.000014,0.000059,0.000092,0.000015


In [21]:
output.to_csv('test_submission.csv',index=False)